In [3]:
import json
import os
from typing import Dict, List, Optional, Tuple, Union

import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import generate, load
from mlx_lm.tuner import TrainingArgs, datasets, linear_to_lora_layers, train
from transformers import PreTrainedTokenizer

In [4]:
model_path = "mlx-community/Phi-3.5-mini-instruct-4bit"
model, tokenizer = load(model_path)

Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 133731.43it/s]


In [5]:
adapter_path = "adapters"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

# Lora config

In [6]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 8,
        "scale": 20.0,
        "dropout": 0.0,
    },
}

In [7]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)

In [8]:
training_args = TrainingArgs(
    adapter_file=adapter_file_path,
    iters=100,
    steps_per_eval=50,
)

In [9]:
model.freeze()
linear_to_lora_layers(model, lora_config["num_layers"], lora_config["lora_parameters"])
num_train_params = sum(v.size for _, v in tree_flatten(model.trainable_parameters()))
print(f"Number of trainable parameters: {num_train_params}")
model.train()

Number of trainable parameters: 786432


Model(
  (model): Phi3Model(
    (embed_tokens): QuantizedEmbedding(32064, 3072, group_size=64, bits=4)
    (layers.0): TransformerBlock(
      (self_attn): Attention(
        (qkv_proj): QuantizedLinear(input_dims=3072, output_dims=9216, bias=False, group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=3072, output_dims=3072, bias=False, group_size=64, bits=4)
        (rope): SuScaledRoPE()
      )
      (mlp): MLP(
        (gate_up_proj): QuantizedLinear(input_dims=3072, output_dims=16384, bias=False, group_size=64, bits=4)
        (down_proj): QuantizedLinear(input_dims=8192, output_dims=3072, bias=False, group_size=64, bits=4)
      )
      (input_layernorm): RMSNorm(3072, eps=1e-05)
      (post_attention_layernorm): RMSNorm(3072, eps=1e-05)
    )
    (layers.1): TransformerBlock(
      (self_attn): Attention(
        (qkv_proj): QuantizedLinear(input_dims=3072, output_dims=9216, bias=False, group_size=64, bits=4)
        (o_proj): QuantizedLinear(input_dims=3072, out

In [10]:
class Metrics:
    def __init__(self) -> None:
        self.train_losses: List[Tuple[int, float]] = []
        self.val_losses: List[Tuple[int, float]] = []

    def on_train_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.train_losses.append((info["iteration"], info["train_loss"]))

    def on_val_loss_report(self, info: Dict[str, Union[float, int]]) -> None:
        self.val_losses.append((info["iteration"], info["val_loss"]))

In [11]:
metrics = Metrics()

# load data

In [12]:
from mlx_lm.tuner.datasets import load_hf_dataset
#config = {
#        'prompt_feature': 'instruction',
#        'completion_feature': 'response',
#}
config = { }
train_set, val_set, test_set = load_hf_dataset(
    data_id="mlx-community/wikisql",
    tokenizer=tokenizer,
    config=config,
)

In [13]:
print(f"Test set size: {len(test_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Training set size: {len(train_set)}")
print(f"test set: {test_set[:2]}")

Test set size: 100
Validation set size: 100
Training set size: 1000
test set: {'text': ["table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What is terrence ross' nationality\nA: SELECT Nationality FROM 1-10015132-16 WHERE Player = 'Terrence Ross'", "table: 1-10015132-16\ncolumns: Player, No., Nationality, Position, Years in Toronto, School/Club Team\nQ: What clu was in toronto 1995-96\nA: SELECT School/Club Team FROM 1-10015132-16 WHERE Years in Toronto = '1995-96'"]}


In [14]:
from mlx_lm.tuner.datasets import CacheDataset

train_dataset = CacheDataset(train_set)
val_dataset = CacheDataset(val_set)

train(
    model,
    optim.Adam(learning_rate=1e-5),
    train_dataset,
    val_dataset,
    args=training_args,
    training_callback=metrics
)


Starting training..., iters: 100


Calculating loss...: 100%|██████████| 25/25 [00:39<00:00,  1.56s/it]

Iter 1: Val loss 3.253, Val took 39.105s


Iter 10: Train loss 3.238, Learning Rate 1.000e-05, It/sec 0.512, Tokens/sec 200.084, Trained Tokens 3911, Peak mem 3.795 GB
Iter 20: Train loss 2.545, Learning Rate 1.000e-05, It/sec 0.566, Tokens/sec 209.774, Trained Tokens 7614, Peak mem 3.795 GB
Iter 30: Train loss 2.185, Learning Rate 1.000e-05, It/sec 0.502, Tokens/sec 196.135, Trained Tokens 11519, Peak mem 3.795 GB
Iter 40: Train loss 1.974, Learning Rate 1.000e-05, It/sec 0.491, Tokens/sec 188.084, Trained Tokens 15352, Peak mem 3.795 GB


Calculating loss...: 100%|██████████| 25/25 [01:03<00:00,  2.54s/it]

Iter 50: Val loss 1.644, Val took 63.529s


Iter 50: Train loss 1.732, Learning Rate 1.000e-05, It/sec 0.423, Tokens/sec 163.246, Trained Tokens 19215, Peak mem 3.795 GB
Iter 60: Train loss 1.515, Learning Rate 1.000e-05, It/sec 0.332, Tokens/sec 130.737, Trained Tokens 23151, Peak mem 3.795 GB
Iter 70: Train loss 1.649, Learning Rate 1.000e-05, It/sec 0.378, Tokens/sec 137.984, Trained Tokens 26801, Peak mem 3.795 GB
Iter 80: Train loss 1.420, Learning Rate 1.000e-05, It/sec 0.414, Tokens/sec 149.610, Trained Tokens 30412, Peak mem 3.795 GB
Iter 90: Train loss 1.392, Learning Rate 1.000e-05, It/sec 0.368, Tokens/sec 142.425, Trained Tokens 34285, Peak mem 3.795 GB


Calculating loss...: 100%|██████████| 25/25 [00:56<00:00,  2.27s/it]

Iter 100: Val loss 1.425, Val took 56.754s


Iter 100: Train loss 1.357, Learning Rate 1.000e-05, It/sec 0.372, Tokens/sec 145.712, Trained Tokens 38202, Peak mem 3.795 GB
Iter 100: Saved adapter weights to adapters/adapters.safetensors and adapters/0000100_adapters.safetensors.
Saved final weights to adapters/adapters.safetensors.


## Fusionar modelo base con adaptador

In [3]:
! python -m mlx_lm fuse  --model ./phi3-mlx --adapter-path ./adapters --save-path ./new_phi3-mlx

Loading pretrained model


In [4]:
! pip install huggingface_hub
! pip install ipywidgets

# Subir modelo a HF

In [5]:
from huggingface_hub import login

login()


In [6]:
from huggingface_hub import create_repo, upload_folder

repo_id = "deimagjas/Phi-3.5-mini-instruct-4bit-sft"

upload_folder(
    folder_path="./new_phi3-mlx",  # carpeta con los archivos del modelo
    repo_id=repo_id
)


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...ning/new_phi3-mlx/model.safetensors:   7%|7         |  151MB / 2.15GB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/deimagjas/Phi-3.5-mini-instruct-4bit-sft/commit/35fb7a075dfd08a9cf05bfea4df0c77929d8fe5e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='35fb7a075dfd08a9cf05bfea4df0c77929d8fe5e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/deimagjas/Phi-3.5-mini-instruct-4bit-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='deimagjas/Phi-3.5-mini-instruct-4bit-sft'), pr_revision=None, pr_num=None)